##  使用z3来做简单证明

### 简单恒等式的证明


- x和y的异或运算，可以用四个机器指令来完成:


![New Project](figs/oxr_operation.png)


- 那么如何证明这四个指令计算的结果和异或运算的结果相同呢？

In [57]:
from z3 import *

x = BitVec('x', 32)
y = BitVec('y', 32)
output = BitVec('output ', 32)
s = Solver ()
s.add(x^y== output)
s.add (((y&x)*0xfffffffe)+(y+x)!= output)
print(s.check())

unsat


### 这是一个暴力枚举证明:
- 遍历所有32位机器数，寻找使得不等式成立的案例(即我们查找的反例)
- 扩展到64位，依旧找不到反例
- 那么，对于所有的数，结论都成立吗？

In [51]:
#!/ usr/bin/python
from z3 import *
x = BitVec('x', 64)
y = BitVec('y', 64)
output = BitVec('output ', 64)
s = Solver ()
s.add(x^y== output)
s.add (((y&x)*0xfffffffffffffffe)+(y+x)!= output)
print (s.check ())

unsat


### 总结:证明一个等式的方法
- 遍历变量的所有可能情况
- 计算等式左边和右边的值
- 检查是否存在 "左边 != 右边" 的情况

用同样的方法，我们证明异或运算还可以用(x + y - ((x & y) <<1))来得到

In [56]:
#!/ usr/bin/python
from z3 import *
x = BitVec('x', 64)
y = BitVec('y', 64)
output = BitVec('output ', 64)
s = Solver ()
s.add(x^y== output)
s.add((x + y - ((x & y) <<1)) != output)
print (s.check ())

unsat


### 使用SAT的证明
![New Project](figs/sat.png)
代码比较复杂，不贴出来了

## 证明特定算法
### sorting network
简单介绍一下sorting network
![New Project](figs/exsorting.png)

下面这个sorting network可以实现排序嘛？
![New Project](figs/sorting_network.png)

In [85]:
from z3 import *
a, b, c, d, e, f, g, h, i=Ints('a b c d e f g h i')

def Z3_min (a, b):
    return If(a<b, a, b)

def Z3_max (a, b):
    return If(a>b, a, b)

def comparator (a, b):
    return (Z3_min(a, b), Z3_max(a, b))

def line(lst , params):
    rt=lst
    start =0
    while start +1 < len(params):
        try:
            first=params.index("+", start)
        except ValueError:
            # no more "+" in parameter string
            return rt
        second=params.index("+", first +1)
        rt[first], rt[second ]= comparator(lst[first], lst[second ])
        start=second +1
    # parameter string ended
    return rt

l=[i, h, g, f, e, d, c, b, a]
l=line(l, " ++++++++")
l=line(l, " + + + + ")
l=line(l, "   +   + ")
l=line(l, " +   +   ")
l=line(l, "+      + ")
l=line(l, "  + + + +")
l=line(l, "    +   +")
l=line(l, "  +   +  ")
l=line(l, "    + +  ")
l=line(l, "   + +++ ")
l=line(l, "+   +    ")
l=line(l, "+ + + +  ")
l=line(l, "+  +     ")
l=line(l, "  +  +   ")
l=line(l, "++++++ ++")

# construct expression like And(..., k[2]>=k[1], k[1]>=k[0])
expr =[(l[k+1]>=l[k]) for k in range(len(l) -1)]
# True if everything works correctly:
correct=And(* expr)
s=Solver ()
# we want to find inputs for which correct == False:
s.add(Not(correct))
print (s.check ()) # must be unsat

unsat


书中后续的几个小证明都是类似的验证布尔表达式的恒等关系的证明。

关于证明的方法这里不再赘述


## 证明中的小技巧

## If-else分支的证明

!(a || b) ? h : !(a == b) ? f : g

!(!a || !b) ? g : (!a && !b) ? h : f

这里用If(a, b, c)来表示 if (a) then (b) else (c)

In [24]:
#!/ usr/bin/python
from z3 import *

a = bool('a')
b = bool('b')
h = bool('h')
f = bool('f')
g = bool('g')
output1 = bool('output1')
output2 = bool('output2')

s = Solver()
s.add(output1 == If(Not(Or(a,b)), h, If( Not(a==b), f, g ) ))
s.add(output2 == If( Not(Or(Not(a),Not(b)) ), g, If( And( Not(a),Not(b)), h, f) )) 
s.add(output1 != output2)
print(s.check())

unsat


## 关于溢出的处理
- 求两个64位机器数的平均值，我们编程实现为:

![New Project](figs/64sum.png)

- 我们尝试用这样的一组机器指令来完成运算，怎么证明正确与否？

![New Project](figs/sum.png)

In [78]:
#!/ usr/bin/python
from z3 import *
x = BitVec('x', 65)
y = BitVec('y', 65)
output = BitVec('output',65)

long_x = BitVec('long_x', 65)
long_y = BitVec('long_y', 65)
sum = BitVec('sum', 65)

s = Solver()
s.add(ULT(x,0xffffffffffffffff),ULT(y,0xffffffffffffffff))
s.add(output == (x + y) >> 1)
s.add((((y^x)>>1)+(y & x)) != output)
print(s.check())
print(s.model())


sat
[y = 18446744073707600047,
 x = 7762023,
 output = 27670116110567232651]


## 封装操作：上取整和下取整
- floor(x + y) != floor(x) + floor(y)
- ceiling(x + y) != ceiling(x) + ceiling(y)


In [34]:
from z3 import *
# Find numbers x and y such that floor(x + y) != floor(x) + floor(y) and
# ceiling(x + y) != ceiling(x) + ceiling(y).
def floor(x):
    return (x&0xff00)
def ceiling(x):
    return If((x&0xff)!=0, (x&0xff00)+0x100 , x)

s=Solver ()
x,y = BitVecs('x y', 16)
s.add(floor(x+y) != floor(x) + floor(y))
s.add(ceiling(x+y) != ceiling(x) + ceiling(y))
print (s.check ())
m=s.model ()
print ("x=0x%04x or %f" % (m[x]. as_long (), float(m[x]. as_long ())/0x100))
print ("y=0x%04x or %f" % (m[y]. as_long (), float(m[y]. as_long ())/0x100))

sat
x=0xbf80 or 191.500000
y=0x0080 or 0.500000


## 等价关系的证明
在z3中，"If and only if" 使用 "==" 来表示；"If any" 使用 "!=" 来体现

"If any" 不能被满足，从而证明"If and only if"

In [36]:
from z3 import *
"""
Prove each of the following statements about inequalities with the floor and ceiling ,
where x is a real number and n is an integer.
a. floor(x) < n iff x < n.
b. n < ceiling(x) iff n < x.
c. n <= floor(x) iff n <= x.
d. floor(x) <= n iff x <= n.
"""
def floor(x):
    return x&0xff00
def ceiling(x):
    return If((x&0xff)!=0, (x&0xff00)+0x100 , x)

s=Solver ()
x = BitVec('x', 16)
n = BitVec('n', 16)
s.add((n&0xff)==0) # n is always integer , it has no fraction
# prevent integer overflow , x and n must be positive
s.add(x<0x8000)
s.add(n<0x8000)
s.add(( floor(x) < n) != (x < n)) # a
#s.add((n < ceiling(x)) != (n < x)) # b
#s.add((n <= floor(x)) != (n <= x)) # c
#s.add(( floor(x) <= n) != (x <= n)) # d
# must be unsat for a/b/c/d
print (s.check ())

unsat
